# 팀 프로젝트 : 코로나 시국 전 vs 후 서울 시민 따릉이 이용현황 비교

In [1]:
import pandas as pd
from numpy import nan

In [2]:
# 그래프 그리는 도구 2
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager, rc

In [3]:
# 그래프 그리는 도구 1 : sns.**plot(x='x축컬럼', y='y축컬럼', data=데이터프레임 형태로)
import seaborn as sns
# 단, 한글 표기시 깨지는 문제가 있어서 추가 작업이 필요함.( 한글 폰트 설정에 관한 코드라인 )
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import matplotlib
import platform

path = "C:/Windows/Fonts/malgun.ttf"
if platform.system() == "Windows":
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system()=="Darwin":
    rc('font', family='AppleGothic')
else:
    print("Unknown System")
    
matplotlib.rcParams['axes.unicode_minus'] = False

In [4]:
# 그래프 그리는 도구 3 : 한글 표기가 용이한 대신, 다양한 형태의 그래프를 그리는데 제약이 따름 (ex. 꺽은 선그래프에 평균값, 중앙값, 최대최소 범위 등을 면적으로 나타내는 기능이 없음.)
import plotly
import cufflinks as cf

In [5]:
import pandas as pd
import numpy as np  
%matplotlib inline  
from plotly import __version__ 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()

## 01) 2018년 데이터셋 불러오기 및 전처리

In [6]:
# 2018년 기본 데이터셋 
bike18 = pd.read_csv('./data_set/서울특별시 공공자전거 이용정보(2018년 전월).csv',encoding='cp949')

# 데이터 전처리(성별)
bike18['성별'] = bike18['성별'].fillna('U')
bike18['성별'] = bike18['성별'].str.replace(pat=r' ', repl=r'U', regex=True)
bike18['성별'] = bike18['성별'].map({'m':'M','M':'M',   'f':'F','F':"F",   'U':'Unknown'})
bike18['성별'].unique()

# 주행속력 컬럼 추가하기 : 기존의 컬럼의 값으로 계산하고, 결측치 제거, 이상치 제거
bike18['주행속력'] = ( bike18['이동거리(M)'] / 1000 ) / ( bike18['이동시간(분)'] / 60 )
bike18['주행속력'] = bike18['주행속력'].replace([np.inf, -np.inf], np.nan)

bike18.info()

/Users/works/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning:

Columns (1,7,8) have mixed types.Specify dtype option on import or set low_memory=False.



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393498 entries, 0 to 393497
Data columns (total 12 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   대여일자     393498 non-null  object 
 1   대여소번호    393493 non-null  object 
 2   대여소      393493 non-null  object 
 3   대여구분코드   393404 non-null  object 
 4   성별       393498 non-null  object 
 5   연령대코드    379348 non-null  object 
 6   이용건수     393498 non-null  int64  
 7   운동량      393498 non-null  object 
 8   탄소량      393498 non-null  object 
 9   이동거리(M)  393498 non-null  float64
 10  이동시간(분)  393498 non-null  int64  
 11  주행속력     393327 non-null  float64
dtypes: float64(2), int64(2), object(8)
memory usage: 36.0+ MB


In [7]:
# 2018년 소계 데이터셋
bike18_sum = bike18[['대여일자','이용건수','이동거리(M)','이동시간(분)']].groupby(['대여일자']).sum()
bike18_sum = pd.DataFrame({'대여일자'    : bike18_sum.index, 
                           '총 이용건수' : bike18_sum['이용건수'], 
                           '총 이동거리' : bike18_sum['이동거리(M)'],
                           '총 이용시간' : bike18_sum['이동시간(분)']})
bike18_sum = bike18_sum.reset_index(drop=True)
bike18_sum

,대여일자,총 이용건수,총 이동거리,총 이용시간
0,2018_01,164365,5.179662e+08,3136546
1,2018_02,168738,5.940648e+08,3659153
2,2018_03,462644,2.014093e+09,12825489
3,2018_04,668614,3.127293e+09,19543911
4,2018_05,929170,4.525074e+09,29062611
5,2018_06,1207120,5.675799e+09,35819848
6,2018_07,1100015,5.131763e+09,30023585
7,2018_08,1037505,4.775283e+09,27858611
8,2018_09,1447993,7.319013e+09,45061826
9,2018_10,1420621,6.578883e+09,38923088


In [8]:
# 2018년 평균 데이터셋
bike18_mean = bike18[['대여일자','이용건수','이동거리(M)','이동시간(분)','주행속력']].groupby(['대여일자']).mean()
bike18_mean = pd.DataFrame({'대여일자' : bike18_mean.index, 
                            '평균 이용건수' : bike18_mean['이용건수'], 
                            '평균 이동거리' : bike18_mean['이동거리(M)'],
                            '평균 이용시간' : bike18_mean['이동시간(분)'],
                            '평균 주행속력' : bike18_mean['주행속력']})
bike18_mean = bike18_mean.reset_index(drop=True)
bike18_mean

,대여일자,평균 이용건수,평균 이동거리,평균 이용시간,평균 주행속력
0,2018_01,11.355880,35785.972779,216.702086,10.792240
1,2018_02,10.831119,38132.406445,234.877271,10.346546
2,2018_03,20.748229,90326.193201,575.185622,9.804563
3,2018_04,24.838918,116178.523293,726.053607,10.172261
4,2018_05,30.547720,148767.937338,955.472630,9.883369
5,2018_06,35.197108,165494.500525,1044.432237,9.705739
6,2018_07,27.929086,130294.101214,762.290789,10.807786
7,2018_08,26.427862,121638.461970,709.628891,10.852809
8,2018_09,31.383277,158629.633065,976.653720,10.300258
9,2018_10,30.414940,140851.306843,833.328081,10.729775


## 02) 2019년 데이터셋 불러오기 및 전처리

In [9]:
# 2019년 기본 데이터셋 
bike19 = pd.read_csv('./data_set/서울특별시 공공자전거 이용정보(2019년 전월).csv')

# 데이터 전처리(성별)
bike19['성별'] = bike19['성별'].fillna('U')
bike19['성별'] = bike19['성별'].str.replace(pat=r'\\N', repl=r'U', regex=True)
bike19['성별'] = bike19['성별'].str.replace(pat=r' ', repl=r'U', regex=True)
bike19['성별'] = bike19['성별'].map({'m':'M','M':'M',   'f':'F','F':"F",   'U':'Unknown'})
bike19['성별'].unique()

# 주행속력 컬럼 추가하기 : 기존의 컬럼의 값으로 계산하고, 결측치 제거, 이상치 제거
bike19['주행속력'] = ( bike19['이동거리(M)'] / 1000 ) / ( bike19['이동시간(분)'] / 60 )
bike19['주행속력'] = bike19['주행속력'].replace([np.inf, -np.inf], np.nan)


bike19.head()

,대여일자,대여소번호,대여소,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리(M),이동시간(분),주행속력
0,2019_01,3,중랑센터,일일(회원),M,AGE_003,12,288.87,2.25,9690,117,4.969231
1,2019_01,3,중랑센터,일일(회원),M,AGE_004,8,424.63,3.32,14310,151,5.686093
2,2019_01,3,중랑센터,일일(회원),M,AGE_005,27,4579.48,41.29,177910,542,19.694834
3,2019_01,3,중랑센터,정기,M,AGE_003,17,3438.8,33.58,144730,593,14.643845
4,2019_01,5,상암센터 정비실,일일(회원),M,AGE_004,1,78.31,0.72,3090,33,5.618182


In [10]:
# 2019년 소계 데이터셋
bike19_sum = bike19[['대여일자','이용건수','이동거리(M)','이동시간(분)','주행속력']].groupby(['대여일자']).sum()
bike19_sum = pd.DataFrame({'대여일자'    : bike19_sum.index, 
                           '총 이용건수' : bike19_sum['이용건수'], 
                           '총 이동거리' : bike19_sum['이동거리(M)'],
                           '총 이용시간' : bike19_sum['이동시간(분)']})

bike19_sum = bike19_sum.reset_index(drop=True)
bike19_sum


,대여일자,총 이용건수,총 이동거리,총 이용시간
0,2019_01,495573,1739532860,9459171
1,2019_02,471536,1848382466,10290548
2,2019_03,904788,3736667090,20897989
3,2019_04,1468848,6719384374,39642036
4,2019_05,2151064,10807324110,61348820
5,2019_06,2293879,12206908080,61987844
6,2019_07,2164086,11355109980,54326375
7,2019_08,2101184,11128106970,56553326
8,2019_09,2084222,11662048710,58548585
9,2019_10,2436401,13386448420,64250207


In [11]:
# 2019년 평균 데이터셋
bike19_mean = bike19[['대여일자','이용건수','이동거리(M)','이동시간(분)','주행속력']].groupby(['대여일자']).mean()
bike19_mean = pd.DataFrame({'대여일자' : bike19_mean.index, 
                            '평균 이용건수' : bike19_mean['이용건수'], 
                            '평균 이동거리' : bike19_mean['이동거리(M)'],
                            '평균 이용시간' : bike19_mean['이동시간(분)'],
                            '평균 주행속력' : bike19_mean['주행속력']})
bike19_mean = bike19_mean.reset_index(drop=True)
bike19_mean

,대여일자,평균 이용건수,평균 이동거리,평균 이용시간,평균 주행속력
0,2019_01,14.232833,49959.299808,271.666935,12.790893
1,2019_02,12.756283,50003.583552,278.386257,11.853320
2,2019_03,20.891937,86281.220329,482.543387,11.644873
3,2019_04,29.725943,135984.141299,802.259244,10.592311
4,2019_05,39.981116,200872.162931,1140.270250,11.375073
5,2019_06,34.611005,184182.933188,935.298509,12.442493
6,2019_07,32.837443,172300.350212,824.338422,13.148214
7,2019_08,30.955758,163945.179811,833.173623,12.953380
8,2019_09,30.287321,169469.573639,850.811378,13.048952
9,2019_10,34.650790,190383.690356,913.774224,13.691653


## 03) 2020년 데이터셋 불러오기 및 전처리

In [12]:
# 2020년 기본 데이터 셋
bike20 = pd.read_csv('./data_set/공공자전거 이용정보(2020년 전월).csv',encoding='cp949')

# 데이터 전처리(성별)
bike20['성별'] = bike20['성별'].fillna('U')
bike20['성별'] = bike20['성별'].str.replace(pat=r'\\N', repl=r'U', regex=True)
bike20['성별'] = bike20['성별'].map({'m':'M','M':'M',   'f':'F','F':"F",   'U':'Unknown'})
bike20['성별'].unique()

# 주행속력 컬럼 추가하기 : 기존의 컬럼의 값으로 계산하고, 결측치 제거, 이상치 제거
bike20['주행속력'] = ( bike20['이동거리'] / 1000 ) / ( bike20['이용시간'] / 60 )
bike20['주행속력'] = bike20['주행속력'].replace([np.inf, -np.inf], np.nan)

bike20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948887 entries, 0 to 948886
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   대여일자    948887 non-null  object 
 1   대여소번호   948887 non-null  int64  
 2   대여소명    948887 non-null  object 
 3   대여구분코드  948887 non-null  object 
 4   성별      948887 non-null  object 
 5   연령대코드   948887 non-null  object 
 6   이용건수    948887 non-null  int64  
 7   운동량     948887 non-null  object 
 8   탄소량     948887 non-null  object 
 9   이동거리    948887 non-null  float64
 10  이용시간    948887 non-null  int64  
 11  주행속력    948208 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 86.9+ MB


In [13]:
# 이동거리 컬럼의 데이터의 자료형이 object이다. 숫자형 으로 바꿔주자
bike20['이동거리'] = pd.to_numeric(bike20['이동거리'])
bike20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948887 entries, 0 to 948886
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   대여일자    948887 non-null  object 
 1   대여소번호   948887 non-null  int64  
 2   대여소명    948887 non-null  object 
 3   대여구분코드  948887 non-null  object 
 4   성별      948887 non-null  object 
 5   연령대코드   948887 non-null  object 
 6   이용건수    948887 non-null  int64  
 7   운동량     948887 non-null  object 
 8   탄소량     948887 non-null  object 
 9   이동거리    948887 non-null  float64
 10  이용시간    948887 non-null  int64  
 11  주행속력    948208 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 86.9+ MB


In [14]:
# 2020년 소계 데이터셋
bike20_sum = bike20[['대여일자','이용건수','이동거리','이용시간']].groupby(['대여일자']).sum()
bike20_sum = pd.DataFrame({'대여일자'   : bike20_sum.index, 
                           '총 이용건수' : bike20_sum['이용건수'], 
                           '총 이동거리' : bike20_sum['이동거리'],
                           '총 이용시간' : bike20_sum['이용시간']})
bike20_sum = bike20_sum.reset_index(drop=True)
bike20_sum

,대여일자,총 이용건수,총 이동거리,총 이용시간
0,2020_01,794162,3.306376e+09,16246705
1,2020_02,762274,3.449431e+09,17374846
2,2020_03,1533535,8.977457e+09,52080367
3,2020_04,2157390,9.810130e+09,74176964
4,2020_05,2480330,9.431412e+09,86465355
5,2020_06,2770773,1.995336e+10,86327378
6,2020_07,2581339,6.636208e+09,79624043
7,2020_08,1731346,1.217681e+09,49392856
8,2020_09,2805248,2.398260e+09,96967938
9,2020_10,2884288,1.283814e+09,86423118


In [15]:
# 2020년 평균 데이터셋
bike20_mean = bike20[['대여일자','이용건수','이동거리','이용시간','주행속력']].groupby(['대여일자']).mean()
bike20_mean = pd.DataFrame({'대여일자'   : bike20_mean.index, 
                            '평균 이용건수' : bike20_mean['이용건수'], 
                            '평균 이동거리' : bike20_mean['이동거리'],
                            '평균 이용시간' : bike20_mean['이용시간'],
                            '평균 주행속력' : bike20_mean['주행속력']})
bike20_mean = bike20_mean.reset_index(drop=True)
bike20_mean

,대여일자,평균 이용건수,평균 이동거리,평균 이용시간,평균 주행속력
0,2020_01,14.859426,61865.014501,303.989241,14.221501
1,2020_02,14.097128,63792.122034,321.322028,13.530483
2,2020_03,22.391295,131080.735445,760.430543,11.637198
3,2020_04,29.310771,133282.559372,1007.784414,8.932365
4,2020_05,31.954368,121505.930364,1113.942812,7.211297
5,2020_06,30.964931,222989.931939,964.756518,15.565551
6,2020_07,28.067185,72156.228003,865.761042,4.942993
7,2020_08,20.440680,14376.231603,583.143717,1.666090
8,2020_09,28.923959,24727.644303,999.803458,1.590456
9,2020_10,29.733395,13234.510099,890.914056,0.937092


## 04) 2021년 데이터셋 불러오기 및 전처리

In [16]:
# 2021년 기본 데이터 셋
bike21 = pd.read_csv('./data_set/공공자전거 이용정보(2021년 전월).csv',encoding='cp949')

# 데이터 전처리(성별)
bike21['성별'] = bike21['성별'].fillna('U')
bike21['성별'] = bike21['성별'].str.replace(pat=r'\\N', repl=r'U', regex=True)
bike21['성별'] = bike21['성별'].map({'m':'M','M':'M',   'f':'F','F':"F",   'U':'Unknown'})
bike21['성별'].unique()

# 주행속력 컬럼 추가하기 : 기존의 컬럼의 값으로 계산하고, 결측치 제거, 이상치 제거
bike21['주행속력'] = ( bike21['이동거리'] / 1000 ) / ( bike21['이용시간'] / 60 )
bike21['주행속력'] = bike21['주행속력'].replace([np.inf, -np.inf], np.nan)

bike21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608016 entries, 0 to 608015
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   대여일자    608016 non-null  object 
 1   대여소번호   608016 non-null  int64  
 2   대여소명    608016 non-null  object 
 3   대여구분코드  608016 non-null  object 
 4   성별      608016 non-null  object 
 5   연령대코드   608016 non-null  object 
 6   이용건수    608016 non-null  int64  
 7   운동량     608016 non-null  object 
 8   탄소량     608016 non-null  object 
 9   이동거리    608016 non-null  float64
 10  이용시간    608016 non-null  int64  
 11  주행속력    607946 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 55.7+ MB


In [17]:
# 2021년 소계 데이터셋
bike21_sum = bike21[['대여일자','이용건수','이동거리','이용시간']].groupby(['대여일자']).sum()
bike21_sum = pd.DataFrame({'대여일자'   : bike21_sum.index, 
                           '총 이용건수' : bike21_sum['이용건수'], 
                           '총 이동거리' : bike21_sum['이동거리'],
                           '총 이용시간' : bike21_sum['이용시간']})
bike21_sum = bike21_sum.append({'대여일자': '2021_07','총 이용건수': nan,'총 이동거리': nan,'총 이용시간': nan}, ignore_index=True)
bike21_sum = bike21_sum.append({'대여일자': '2021_08','총 이용건수': nan,'총 이동거리': nan,'총 이용시간': nan}, ignore_index=True)
bike21_sum = bike21_sum.append({'대여일자': '2021_09','총 이용건수': nan,'총 이동거리': nan,'총 이용시간': nan}, ignore_index=True)
bike21_sum = bike21_sum.append({'대여일자': '2021_10','총 이용건수': nan,'총 이동거리': nan,'총 이용시간': nan}, ignore_index=True)
bike21_sum = bike21_sum.append({'대여일자': '2021_11','총 이용건수': nan,'총 이동거리': nan,'총 이용시간': nan}, ignore_index=True)
bike21_sum = bike21_sum.append({'대여일자': '2021_12','총 이용건수': nan,'총 이동거리': nan,'총 이용시간': nan}, ignore_index=True)


bike21_sum = bike21_sum.reset_index(drop=True)
bike21_sum

,대여일자,총 이용건수,총 이동거리,총 이용시간
0,2021_01,792606.0,2.201964e+09,20400822.0
1,2021_02,1233682.0,4.139912e+09,35840830.0
2,2021_03,2182801.0,6.864851e+09,59106536.0
3,2021_04,3041378.0,9.753858e+09,86005299.0
4,2021_05,2948438.0,9.601397e+09,82650789.0
5,2021_06,3458394.0,1.054717e+10,89796082.0
6,2021_07,NaN,NaN,NaN
7,2021_08,NaN,NaN,NaN
8,2021_09,NaN,NaN,NaN
9,2021_10,NaN,NaN,NaN


In [18]:
# 2021년 평균 데이터셋
bike21_mean = bike21[['대여일자','이용건수','이동거리','이용시간','주행속력']].groupby(['대여일자']).mean()
bike21_mean = pd.DataFrame({'대여일자'   : bike21_mean.index, 
                            '평균 이용건수' : bike21_mean['이용건수'], 
                            '평균 이동거리' : bike21_mean['이동거리'],
                            '평균 이용시간' : bike21_mean['이용시간'],
                            '평균 주행속력' : bike21_mean['주행속력']})
bike21_mean = bike21_mean.append({'대여일자': '2021_07','평균 이용건수': nan,'평균 이동거리': nan,'평균 이용시간': nan, '평균 주행속력':nan}, ignore_index=True)
bike21_mean = bike21_mean.append({'대여일자': '2021_08','평균 이용건수': nan,'평균 이동거리': nan,'평균 이용시간': nan, '평균 주행속력':nan}, ignore_index=True)
bike21_mean = bike21_mean.append({'대여일자': '2021_09','평균 이용건수': nan,'평균 이동거리': nan,'평균 이용시간': nan, '평균 주행속력':nan}, ignore_index=True)
bike21_mean = bike21_mean.append({'대여일자': '2021_10','평균 이용건수': nan,'평균 이동거리': nan,'평균 이용시간': nan, '평균 주행속력':nan}, ignore_index=True)
bike21_mean = bike21_mean.append({'대여일자': '2021_11','평균 이용건수': nan,'평균 이동거리': nan,'평균 이용시간': nan, '평균 주행속력':nan}, ignore_index=True)
bike21_mean = bike21_mean.append({'대여일자': '2021_12','평균 이용건수': nan,'평균 이동거리': nan,'평균 이용시간': nan, '평균 주행속력':nan}, ignore_index=True)


bike21_mean = bike21_mean.reset_index(drop=True)
bike21_mean

,대여일자,평균 이용건수,평균 이동거리,평균 이용시간,평균 주행속력
0,2021_01,11.532002,32037.415714,296.821260,6.680658
1,2021_02,14.232110,47759.210542,413.470115,7.061745
2,2021_03,21.825391,68640.269269,590.994441,6.930793
3,2021_04,26.505769,85005.384854,749.540708,6.766645
4,2021_05,24.978084,81339.509959,700.187130,6.956710
5,2021_06,28.866859,88036.125808,749.518651,7.073855
6,2021_07,NaN,NaN,NaN,NaN
7,2021_08,NaN,NaN,NaN,NaN
8,2021_09,NaN,NaN,NaN,NaN
9,2021_10,NaN,NaN,NaN,NaN


## 05) 18년 ~ 21년 총 이용건수 비교

In [49]:
bike_merge_sum = pd.DataFrame({'Month'   : ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], 
                               '18년 총 이용건수' : bike18_sum['총 이용건수'],
                               '19년 총 이용건수' : bike19_sum['총 이용건수'], 
                               '20년 총 이용건수' : bike20_sum['총 이용건수'],
                               '21년 총 이용건수' : bike21_sum['총 이용건수']})
bike_merge_sum = bike_merge_sum.reset_index(drop=True)
bike_merge_sum 

,Month,18년 총 이용건수,19년 총 이용건수,20년 총 이용건수,21년 총 이용건수
0,Jan,164365,495573,794162,792606.0
1,Feb,168738,471536,762274,1233682.0
2,Mar,462644,904788,1533535,2182801.0
3,Apr,668614,1468848,2157390,3041378.0
4,May,929170,2151064,2480330,2948438.0
5,Jun,1207120,2293879,2770773,3458394.0
6,Jul,1100015,2164086,2581339,NaN
7,Aug,1037505,2101184,1731346,NaN
8,Sep,1447993,2084222,2805248,NaN
9,Oct,1420621,2436401,2884288,NaN


In [51]:
bike_merge_sum.iplot(kind='bar',x='Month',theme='space',title="2018년~2021년 총 이용건수 비교", xTitle="월", yTitle="총 이용건수")

In [52]:
bike_merge_sum.iplot(kind='line',x='Month',theme='space',title="2018년~2021년 총 이용건수 비교", xTitle="월", yTitle="총 이용건수")

## 07) 18년 ~ 21년 총 이용시간 비교

In [22]:
bike_merge_sum2 = pd.DataFrame({'Month'   : ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], 
                               '18년 총 이용시간' : bike18_sum['총 이용시간'],
                               '19년 총 이용시간' : bike19_sum['총 이용시간'], 
                               '20년 총 이용시간' : bike20_sum['총 이용시간'],
                               '21년 총 이용시간' : bike21_sum['총 이용시간']})

bike_merge_sum2 = bike_merge_sum2.reset_index(drop=True)
bike_merge_sum2 

,Month,18_총 이용시간,19_총 이용시간,20_총 이용시간,21_총 이용시간
0,Jan,3136546,9459171,16246705,20400822.0
1,Feb,3659153,10290548,17374846,35840830.0
2,Mar,12825489,20897989,52080367,59106536.0
3,Apr,19543911,39642036,74176964,86005299.0
4,May,29062611,61348820,86465355,82650789.0
5,Jun,35819848,61987844,86327378,89796082.0
6,Jul,30023585,54326375,79624043,NaN
7,Aug,27858611,56553326,49392856,NaN
8,Sep,45061826,58548585,96967938,NaN
9,Oct,38923088,64250207,86423118,NaN


In [23]:
bike_merge_sum2.iplot(kind='line',x='Month',theme='space',title="월별 총 이용시간 연간 비교", xTitle="월", yTitle="총 이용시간")

## 08) 18년 ~ 21년 총 이동거리 비교

In [24]:
bike_merge_sum3 = pd.DataFrame({'Month'   : ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], 
                               '18년 총 이동거리' : bike18_sum['총 이동거리'],
                               '19년 총 이동거리' : bike19_sum['총 이동거리'], 
                               '20년 총 이동거리' : bike20_sum['총 이동거리'],
                               '21년 총 이동거리' : bike21_sum['총 이동거리']})

bike_merge_sum3 = bike_merge_sum3.reset_index(drop=True)
bike_merge_sum3

,Month,18_총 이동거리,19_총 이동거리,20_총 이동거리,21_총 이동거리
0,Jan,5.179662e+08,1739532860,3.306376e+09,2.201964e+09
1,Feb,5.940648e+08,1848382466,3.449431e+09,4.139912e+09
2,Mar,2.014093e+09,3736667090,8.977457e+09,6.864851e+09
3,Apr,3.127293e+09,6719384374,9.810130e+09,9.753858e+09
4,May,4.525074e+09,10807324110,9.431412e+09,9.601397e+09
5,Jun,5.675799e+09,12206908080,1.995336e+10,1.054717e+10
6,Jul,5.131763e+09,11355109980,6.636208e+09,NaN
7,Aug,4.775283e+09,11128106970,1.217681e+09,NaN
8,Sep,7.319013e+09,11662048710,2.398260e+09,NaN
9,Oct,6.578883e+09,13386448420,1.283814e+09,NaN


In [25]:
bike_merge_sum3.iplot(kind='line',x='Month',theme='space')

## 09) 18년 ~ 21년 한 사람의 평균 이용건수 비교

In [26]:
bike_merge_mean = pd.DataFrame({'Month'   : ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], 
                               '18_한사람 평균 이용건수' : bike18_mean['평균 이용건수'],
                               '19_한사람 평균 이용건수' : bike19_mean['평균 이용건수'], 
                               '20_한사람 평균 이용건수' : bike20_mean['평균 이용건수'],
                               '21_한사람 평균 이용건수' : bike21_mean['평균 이용건수']})
bike_merge_mean = bike_merge_mean.reset_index(drop=True)
bike_merge_mean

,Month,18_한사람 평균 이용건수,19_한사람 평균 이용건수,20_한사람 평균 이용건수,21_한사람 평균 이용건수
0,Jan,11.355880,14.232833,14.859426,11.532002
1,Feb,10.831119,12.756283,14.097128,14.232110
2,Mar,20.748229,20.891937,22.391295,21.825391
3,Apr,24.838918,29.725943,29.310771,26.505769
4,May,30.547720,39.981116,31.954368,24.978084
5,Jun,35.197108,34.611005,30.964931,28.866859
6,Jul,27.929086,32.837443,28.067185,NaN
7,Aug,26.427862,30.955758,20.440680,NaN
8,Sep,31.383277,30.287321,28.923959,NaN
9,Oct,30.414940,34.650790,29.733395,NaN


In [27]:
bike_merge_mean.iplot(kind='line',x='Month',theme='space')

## 10) 18년 ~ 21년 한 사람의 평균 이용시간 비교

In [28]:
bike_merge_mean2 = pd.DataFrame({'Month'   : ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], 
                               '18_한사람 평균 이용시간' : bike18_mean['평균 이용시간'],
                               '19_한사람 평균 이용시간' : bike19_mean['평균 이용시간'], 
                               '20_한사람 평균 이용시간' : bike20_mean['평균 이용시간'],
                               '21_한사람 평균 이용시간' : bike21_mean['평균 이용시간']})
bike_merge_mean2 = bike_merge_mean2.reset_index(drop=True)
bike_merge_mean2

,Month,18_한사람 평균 이용시간,19_한사람 평균 이용시간,20_한사람 평균 이용시간,21_한사람 평균 이용시간
0,Jan,216.702086,271.666935,303.989241,296.821260
1,Feb,234.877271,278.386257,321.322028,413.470115
2,Mar,575.185622,482.543387,760.430543,590.994441
3,Apr,726.053607,802.259244,1007.784414,749.540708
4,May,955.472630,1140.270250,1113.942812,700.187130
5,Jun,1044.432237,935.298509,964.756518,749.518651
6,Jul,762.290789,824.338422,865.761042,NaN
7,Aug,709.628891,833.173623,583.143717,NaN
8,Sep,976.653720,850.811378,999.803458,NaN
9,Oct,833.328081,913.774224,890.914056,NaN


In [29]:
bike_merge_mean2.iplot(kind='line',x='Month',theme='space')

## 11) 18년 ~ 21년 한 사람의 평균 이동거리 비교

In [30]:
bike_merge_mean3 = pd.DataFrame({'Month'   : ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], 
                               '18_한사람 평균 이동거리' : bike18_mean['평균 이동거리'],
                               '19_한사람 평균 이동거리' : bike19_mean['평균 이동거리'], 
                               '20_한사람 평균 이동거리' : bike20_mean['평균 이동거리'],
                               '21_한사람 평균 이동거리' : bike21_mean['평균 이동거리']})
bike_merge_mean3 = bike_merge_mean3.reset_index(drop=True)
bike_merge_mean3

,Month,18_한사람 평균 이동거리,19_한사람 평균 이동거리,20_한사람 평균 이동거리,21_한사람 평균 이동거리
0,Jan,35785.972779,49959.299808,61865.014501,32037.415714
1,Feb,38132.406445,50003.583552,63792.122034,47759.210542
2,Mar,90326.193201,86281.220329,131080.735445,68640.269269
3,Apr,116178.523293,135984.141299,133282.559372,85005.384854
4,May,148767.937338,200872.162931,121505.930364,81339.509959
5,Jun,165494.500525,184182.933188,222989.931939,88036.125808
6,Jul,130294.101214,172300.350212,72156.228003,NaN
7,Aug,121638.461970,163945.179811,14376.231603,NaN
8,Sep,158629.633065,169469.573639,24727.644303,NaN
9,Oct,140851.306843,190383.690356,13234.510099,NaN


In [31]:
bike_merge_mean3.iplot(kind='line',x='Month',theme='space')

## 12) 18년 ~ 21년 한 사람의 평균 주행속력 비교

In [32]:
bike_merge_mean4 = pd.DataFrame({'Month'   : ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], 
                               '18년 한사람 평균 주행속력' : bike18_mean['평균 주행속력'],
                               '19년 한사람 평균 주행속력' : bike19_mean['평균 주행속력'], 
                               '20년 한사람 평균 주행속력' : bike20_mean['평균 주행속력'],
                               '21년 한사람 평균 주행속력' : bike21_mean['평균 주행속력']})
bike_merge_mean4 = bike_merge_mean4.reset_index(drop=True)
bike_merge_mean4

,Month,18년 한사람 평균 주행속력,19년 한사람 평균 주행속력,20년 한사람 평균 주행속력,21년 한사람 평균 주행속력
0,Jan,10.792240,12.790893,14.221501,6.680658
1,Feb,10.346546,11.853320,13.530483,7.061745
2,Mar,9.804563,11.644873,11.637198,6.930793
3,Apr,10.172261,10.592311,8.932365,6.766645
4,May,9.883369,11.375073,7.211297,6.956710
5,Jun,9.705739,12.442493,15.565551,7.073855
6,Jul,10.807786,13.148214,4.942993,NaN
7,Aug,10.852809,12.953380,1.666090,NaN
8,Sep,10.300258,13.048952,1.590456,NaN
9,Oct,10.729775,13.691653,0.937092,NaN


In [33]:
bike_merge_mean4.iplot(kind='line',x='Month',theme='space')

## 13) 남녀 성별에 따른 주행속력 비교

In [34]:
bike19_gender = bike19[['성별','주행속력']].groupby(['성별']).mean()
bike19_gender

,주행속력
성별,
F,12.196185
M,13.474503
Unknown,12.718673


In [35]:
bike19_gender = pd.DataFrame({'성별'   : ['Female','Male','Unknown'], 
                              '평균 주행속력' : bike19_gender['주행속력']})

bike19_gender = bike19_gender.reset_index(drop=True)
bike19_gender

,성별,평균 주행속력
0,Female,12.196185
1,Male,13.474503
2,Unknown,12.718673


In [36]:
# unknown은 굳이 넣을 필요없으니까 [0:2] 부분만 출력
bike19_gender[0:2].iplot(kind='bar',x='성별' ,theme='space')

## 14) 2019년 2020년 총 이용건수 증감비율

In [37]:
bike_ratio_sum = pd.DataFrame({'Month'   : ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], 
                               '18년 총 이용건수' : bike18_sum['총 이용건수'],
                               '19년 총 이용건수' : bike19_sum['총 이용건수'], 
                               '20년 총 이용건수' : bike20_sum['총 이용건수'],
                               '21년 총 이용건수' : bike21_sum['총 이용건수']})
bike_ratio_sum = bike_ratio_sum.reset_index(drop=True)
bike_ratio_sum

,Month,18년 총 이용건수,19년 총 이용건수,20년 총 이용건수,21년 총 이용건수
0,Jan,164365,495573,794162,792606.0
1,Feb,168738,471536,762274,1233682.0
2,Mar,462644,904788,1533535,2182801.0
3,Apr,668614,1468848,2157390,3041378.0
4,May,929170,2151064,2480330,2948438.0
5,Jun,1207120,2293879,2770773,3458394.0
6,Jul,1100015,2164086,2581339,NaN
7,Aug,1037505,2101184,1731346,NaN
8,Sep,1447993,2084222,2805248,NaN
9,Oct,1420621,2436401,2884288,NaN


In [38]:
bike_ratio_sum['18년vs19년'] = ( bike_ratio_sum['19년 총 이용건수']- bike_ratio_sum['18년 총 이용건수']) / bike_ratio_sum['18년 총 이용건수'] * 100
bike_ratio_sum['19년vs20년'] = ( bike_ratio_sum['20년 총 이용건수']- bike_ratio_sum['19년 총 이용건수']) / bike_ratio_sum['19년 총 이용건수'] * 100
bike_ratio_sum['20년vs21년'] = ( bike_ratio_sum['21년 총 이용건수']- bike_ratio_sum['20년 총 이용건수']) / bike_ratio_sum['20년 총 이용건수'] * 100

bike_ratio_sum.head()

,Month,18년 총 이용건수,19년 총 이용건수,20년 총 이용건수,21년 총 이용건수,18년vs19년,19년vs20년,20년vs21년
0,Jan,164365,495573,794162,792606.0,201.507620,60.251265,-0.195930
1,Feb,168738,471536,762274,1233682.0,179.448613,61.657646,61.842330
2,Mar,462644,904788,1533535,2182801.0,95.568947,69.491085,42.337866
3,Apr,668614,1468848,2157390,3041378.0,119.685499,46.876328,40.974882
4,May,929170,2151064,2480330,2948438.0,131.503815,15.307122,18.872811


In [39]:
bike_ratio_sum = bike_ratio_sum.drop(['18년 총 이용건수','19년 총 이용건수','20년 총 이용건수','21년 총 이용건수'],axis=1)
bike_ratio_sum.head()

,Month,18년vs19년,19년vs20년,20년vs21년
0,Jan,201.507620,60.251265,-0.195930
1,Feb,179.448613,61.657646,61.842330
2,Mar,95.568947,69.491085,42.337866
3,Apr,119.685499,46.876328,40.974882
4,May,131.503815,15.307122,18.872811


In [48]:
#bike_ratio_sum.iplot(kind='line',x='Month',theme='space')
bike_ratio_sum.iloc[:,[0,2]].iplot(kind='line',x='Month',theme='space',title="2019년 2020년 총 이용건수 증감비율", xTitle="월", yTitle="작년 대비 총 이용건수 증감 비율(%)")

In [41]:
#bike_ratio_sum.iplot(kind='bar',x='Month',theme='space')
bike_ratio_sum.iloc[:,[0,2]].iplot(kind='bar',x='Month',theme='space',title="2019년 2020년 총 이용건수 증감비율", xTitle="월", yTitle="작년 대비 총 이용건수 증감 비율(%)")

In [42]:
bike_ratio_sum.iloc[:,[0,1]].iplot(kind='bar',x='Month',theme='space',title="2018년 2019년 총 이용건수 증감비율", xTitle="월", yTitle="작년 대비 총 이용건수 증감 비율(%)")

In [43]:
bike_ratio_sum.iloc[:,[0,1,2]].iplot(kind='bar',x='Month',theme='space',title="총 이용건수 증감비율", xTitle="월", yTitle="작년 대비 총 이용건수 증감 비율(%)")

## 15) 2019년 2020년 총 이용시간 증감비율

## 16) 2019년 2020년 총 이동거리 증감비율

## 17) 2019년 2020년 한 사람 평균 이용건수 증감비율

## 18) 2019년 2020년 한 사람 평균 이용시간 증감비율

## 19) 2019년 2020년 한 사람 평균 이동거리 증감비율

## 20) 2019년 2020년 한 사람 평균 주행속력 증감비율

In [44]:
bike_ratio_mean4 = pd.DataFrame({'Month'   : ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], 
                               '18년 평균 주행속력' : bike18_mean['평균 주행속력'],
                               '19년 평균 주행속력' : bike19_mean['평균 주행속력'], 
                               '20년 평균 주행속력' : bike20_mean['평균 주행속력'],
                               '21년 평균 주행속력' : bike21_mean['평균 주행속력']})

bike_ratio_mean4 = bike_ratio_mean4.reset_index(drop=True)
bike_ratio_mean4

,Month,18년 평균 주행속력,19년 평균 주행속력,20년 평균 주행속력,21년 평균 주행속력
0,Jan,10.792240,12.790893,14.221501,6.680658
1,Feb,10.346546,11.853320,13.530483,7.061745
2,Mar,9.804563,11.644873,11.637198,6.930793
3,Apr,10.172261,10.592311,8.932365,6.766645
4,May,9.883369,11.375073,7.211297,6.956710
5,Jun,9.705739,12.442493,15.565551,7.073855
6,Jul,10.807786,13.148214,4.942993,NaN
7,Aug,10.852809,12.953380,1.666090,NaN
8,Sep,10.300258,13.048952,1.590456,NaN
9,Oct,10.729775,13.691653,0.937092,NaN


In [45]:
bike_ratio_mean4['18년vs19년'] = ( bike_ratio_mean4['19년 평균 주행속력']- bike_ratio_mean4['18년 평균 주행속력']) / bike_ratio_mean4['18년 평균 주행속력'] * 100
bike_ratio_mean4['19년vs20년'] = ( bike_ratio_mean4['20년 평균 주행속력']- bike_ratio_mean4['19년 평균 주행속력']) / bike_ratio_mean4['19년 평균 주행속력'] * 100
bike_ratio_mean4['20년vs21년'] = ( bike_ratio_mean4['21년 평균 주행속력']- bike_ratio_mean4['20년 평균 주행속력']) / bike_ratio_mean4['20년 평균 주행속력'] * 100

bike_ratio_mean4.head()

,Month,18년 평균 주행속력,19년 평균 주행속력,20년 평균 주행속력,21년 평균 주행속력,18년vs19년,19년vs20년,20년vs21년
0,Jan,10.792240,12.790893,14.221501,6.680658,18.519353,11.184587,-53.024241
1,Feb,10.346546,11.853320,13.530483,7.061745,14.563068,14.149305,-47.808628
2,Mar,9.804563,11.644873,11.637198,6.930793,18.769942,-0.065913,-40.442766
3,Apr,10.172261,10.592311,8.932365,6.766645,4.129368,-15.671237,-24.245761
4,May,9.883369,11.375073,7.211297,6.956710,15.093080,-36.604390,-3.530392


In [46]:
bike_ratio_mean4 = bike_ratio_mean4.drop(['18년 평균 주행속력','19년 평균 주행속력','20년 평균 주행속력','21년 평균 주행속력'],axis=1)
bike_ratio_mean4.head()

,Month,18년vs19년,19년vs20년,20년vs21년
0,Jan,18.519353,11.184587,-53.024241
1,Feb,14.563068,14.149305,-47.808628
2,Mar,18.769942,-0.065913,-40.442766
3,Apr,4.129368,-15.671237,-24.245761
4,May,15.093080,-36.604390,-3.530392


In [47]:
bike_ratio_mean4.iloc[:,[0,1]].iplot(kind='bar',x='Month',theme='space',title="2018년 2019년 월별 한 사람의 평균 주행속력 증감비율 연간 비교", xTitle="월", yTitle="평균 주행속력")
#bike_ratio_mean4.iplot(kind='line',x='Month',theme='space',title="월별 한 사람의 평균 주행속력 연간 비교", xTitle="월", yTitle="평균 주행속력")